In [1]:
import pandas as pd
import sqlite3
import warnings
import re
from datetime import datetime
warnings.filterwarnings(action='ignore')

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [3]:
con = sqlite3.connect("../db/hr.db")
df = pd.read_sql_query("SELECT 자료생성년월, 사업장명, 사업자등록번호, 사업장업종코드명, 가입자수, 신규취득자수, 상실가입자수, 당월고지금액  from pension", con)

In [4]:
df['자료생성년월'].unique()

array(['2023-01', '2022-12', '2022-11', '2022-10', '2022-09', '2022-08',
       'Jul-22', 'Jun-22', 'May-22', 'Apr-22', 'Mar-22', 'Feb-22',
       'Jan-22', '2023-02', '2023-03'], dtype=object)

In [5]:
import functools # not required, but helps in production
def unpack_df_columns(func):
    """
    A general use decorator to unpack a df[subset] of columns
    into a function which expects the values at those columns
    as arguments
    """
    
    @functools.wraps(func)
    def _unpack_df_columns(*args, **kwargs):
        
        # args[0] is a pandas series equal in length as the 
        # df[subset] to which the apply function is applied 
        series = args[0]

        # series.values holds the number of arguments expected
        # by func and is os length len(df[subset].columns)
        return func(*series.values)

    return _unpack_df_columns

In [6]:
def date_change(val):
    if val == 'Jul-22':
        return '2022-07'
    elif val == 'Jun-22':
        return '2022-06'
    elif val == 'May-22':
        return '2022-05'
    elif val == 'Apr-22':
        return '2022-04'
    elif val == 'Mar-22':
        return '2022-03'
    elif val == 'Feb-22':
        return '2022-02'
    elif val == 'Jan-22':
        return '2022-01'
    else:
        return val

In [7]:
def 월평균가입자수(가입자수):
    return int(가입자수/12)

In [8]:
@unpack_df_columns
def get_평균소득월액(고지액, 인원):
    평균소득월액 = (int(고지액) / int(인원)) / 0.09
    return int(평균소득월액)

In [9]:
df1 = df.copy()
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8172272 entries, 0 to 8172271
Data columns (total 8 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   자료생성년월    object
 1   사업장명      object
 2   사업자등록번호   int64 
 3   사업장업종코드명  object
 4   가입자수      int64 
 5   신규취득자수    int64 
 6   상실가입자수    int64 
 7   당월고지금액    int64 
dtypes: int64(5), object(3)
memory usage: 498.8+ MB


## 날짜 변환

In [10]:
df1['자료생성년월'] = df1['자료생성년월'].apply(date_change)

In [11]:
df1['자료생성년월'].unique()

array(['2023-01', '2022-12', '2022-11', '2022-10', '2022-09', '2022-08',
       '2022-07', '2022-06', '2022-05', '2022-04', '2022-03', '2022-02',
       '2022-01', '2023-02', '2023-03'], dtype=object)

In [44]:
# gdf = df1.groupby(['사업장업종코드명'])[['가입자수','신규취득자수','상실가입자수']].sum().reset_index()
# gdf

In [45]:
# gdf["사업장업종코드명"].unique().tolist()

In [46]:
# gdf[gdf["사업장업종코드명"] == '건설 및 채광용 기계장비 제조업']

In [47]:
# gdf[gdf["사업장업종코드명"] == '화물 자동차 및 특수 목적용 자동차 제조업']

In [48]:
# df2 = df1[df1["사업장업종코드명"] == '건설 및 채광용 기계장비 제조업'][['자료생성년월', '사업장명','가입자수','신규취득자수','상실가입자수']].sort_values(by="가입자수", ascending=False)
# df2 = df2.groupby(["사업장명"])[['가입자수','신규취득자수','상실가입자수']].sum().reset_index().sort_values(by="가입자수", ascending=False)
# df2["월평균가입자수"] = df2["가입자수"].apply(월평균가입자수)
# df2["상실률"] = df2["상실가입자수"]/df2["월평균가입자수"]
# df2

In [49]:
# df3 = df1[df1["사업장업종코드명"] == '화물 자동차 및 특수 목적용 자동차 제조업'][['자료생성년월', '사업장명','가입자수','신규취득자수','상실가입자수']].sort_values(by="가입자수", ascending=False)
# df3 = df3.groupby(["사업장명"])[['가입자수','신규취득자수','상실가입자수']].sum().reset_index().sort_values(by="가입자수", ascending=False)
# df3["월평균가입자수"] = df3["가입자수"].apply(월평균가입자수)
# df3["상실률"] = df3["상실가입자수"]/df3["월평균가입자수"]
# df3

In [50]:
# df4 = df1[df1["사업장업종코드명"] == '비디오 및 기타 영상 기기 제조업'][['자료생성년월', '사업장명','가입자수','신규취득자수','상실가입자수']].sort_values(by="가입자수", ascending=False)
# df4 = df4.groupby(["사업장명"])[['가입자수','신규취득자수','상실가입자수']].sum().reset_index().sort_values(by="가입자수", ascending=False)
# df4["월평균가입자수"] = df4["가입자수"].apply(월평균가입자수)
# df4["상실률"] = df4["상실가입자수"]/df4["월평균가입자수"]
# df4.head(20)

In [51]:
# df5 = df1[df1["사업장업종코드명"] == '컴퓨터 제조업'][['자료생성년월', '사업장명','가입자수','신규취득자수','상실가입자수']].sort_values(by="가입자수", ascending=False)
# df5 = df5.groupby(["사업장명"])[['가입자수','신규취득자수','상실가입자수']].sum().reset_index().sort_values(by="가입자수", ascending=False)
# df5["월평균가입자수"] = df5["가입자수"].apply(월평균가입자수)
# df5["상실률"] = df5["상실가입자수"]/df5["월평균가입자수"]
# df5.head(20)

In [52]:
# df6 = df1[df1["사업장업종코드명"] == '산업용 로봇 제조업'][['자료생성년월', '사업장명','가입자수','신규취득자수','상실가입자수']].sort_values(by="가입자수", ascending=False)
# df6 = df6.groupby(["사업장명"])[['가입자수','신규취득자수','상실가입자수']].sum().reset_index().sort_values(by="가입자수", ascending=False)
# df6["월평균가입자수"] = df6["가입자수"].apply(월평균가입자수)
# df6["상실률"] = df6["상실가입자수"]/df6["월평균가입자수"]
# df6.head(20)

In [53]:
# df7 = df1[df1["사업장업종코드명"] == '농업 및 임업용 기계 제조업'][['자료생성년월', '사업장명','가입자수','신규취득자수','상실가입자수']].sort_values(by="가입자수", ascending=False)
# df7 = df7.groupby(["사업장명"])[['가입자수','신규취득자수','상실가입자수']].sum().reset_index().sort_values(by="가입자수", ascending=False)
# df7["월평균가입자수"] = df7["가입자수"].apply(월평균가입자수)
# df7["상실률"] = df7["상실가입자수"]/df7["월평균가입자수"]
# df7.head(20)

## 경쟁사 Data Load

In [19]:
comp_df = pd.read_excel("./경쟁사.xlsx")
comp_df

,사업장명,사업자등록번호,약식명
0,현대자동차(주),101810,현대차
1,현대모비스(주),101811,모비스
2,삼성전자(주),124810,삼성전자
3,두산밥캣주식회사,201863,두산밥캣
4,볼보그룹코리아(주),609813,볼보코리아
5,볼보그룹코리아(주)건설기계(서울),120851,볼보코리아
6,두산산업차량주식회사,802880,두산산차
7,(주)대동,514810,대동
8,(주)한양정밀,131810,한양정밀
9,(주)모트롤,263810,모트롤


In [20]:
df2 = df1.loc[(df1["사업장명"].isin(comp_df["사업장명"].tolist())) & (df1["사업자등록번호"].isin(comp_df["사업자등록번호"].tolist()))]

In [21]:
df2

,자료생성년월,사업장명,사업자등록번호,사업장업종코드명,가입자수,신규취득자수,상실가입자수,당월고지금액
1931,2023-01,볼보그룹코리아(주)건설기계(서울),120851,상품 종합 도매업,318,2,3,145372160
3326,2023-01,현대자동차(주),101810,화물 자동차 및 특수 목적용 자동차 제조업,68104,1227,681,32113238560
3674,2023-01,현대모비스(주),101811,화물 자동차 및 특수 목적용 자동차 제조업,11318,137,72,5384272380
4320,2023-01,삼성전자(주),124810,비디오 및 기타 영상 기기 제조업,118094,653,623,57429417660
91866,2023-01,현대두산인프라코어(주),121814,건설 및 채광용 기계장비 제조업,2578,46,16,1223206740
124715,2023-01,엘에스엠트론(주)전주공장,138814,제철업,838,15,18,346643560
125110,2023-01,엘에스엠트론（주）,138814,제철업,100,5,0,46199900
148800,2023-01,(주)대동,514810,농업 및 임업용 기계 제조업,1267,26,40,503290240
151091,2023-01,볼보그룹코리아(주),609813,상품 종합 도매업,1148,1,13,520559900
171356,2023-01,(주)모트롤,263810,BIZ_NO미존재사업장,489,5,1,229944440


In [22]:
df3 = pd.merge(df2,comp_df, on=['사업장명','사업자등록번호'], how='left')

In [23]:
df3

,자료생성년월,사업장명,사업자등록번호,사업장업종코드명,가입자수,신규취득자수,상실가입자수,당월고지금액,약식명
0,2023-01,볼보그룹코리아(주)건설기계(서울),120851,상품 종합 도매업,318,2,3,145372160,볼보코리아
1,2023-01,현대자동차(주),101810,화물 자동차 및 특수 목적용 자동차 제조업,68104,1227,681,32113238560,현대차
2,2023-01,현대모비스(주),101811,화물 자동차 및 특수 목적용 자동차 제조업,11318,137,72,5384272380,모비스
3,2023-01,삼성전자(주),124810,비디오 및 기타 영상 기기 제조업,118094,653,623,57429417660,삼성전자
4,2023-01,현대두산인프라코어(주),121814,건설 및 채광용 기계장비 제조업,2578,46,16,1223206740,HDI
5,2023-01,엘에스엠트론(주)전주공장,138814,제철업,838,15,18,346643560,엠트론
6,2023-01,엘에스엠트론（주）,138814,제철업,100,5,0,46199900,엠트론
7,2023-01,(주)대동,514810,농업 및 임업용 기계 제조업,1267,26,40,503290240,대동
8,2023-01,볼보그룹코리아(주),609813,상품 종합 도매업,1148,1,13,520559900,볼보코리아
9,2023-01,(주)모트롤,263810,BIZ_NO미존재사업장,489,5,1,229944440,모트롤


In [24]:
gdf = df3.groupby(['약식명', '자료생성년월'])[['가입자수', '신규취득자수', '상실가입자수', '당월고지금액']].sum().reset_index()

In [25]:
gdf

,약식명,자료생성년월,가입자수,신규취득자수,상실가입자수,당월고지금액
0,HCE,2022-01,1331,19,15,557290480
1,HCE,2022-02,1357,41,16,566574100
2,HCE,2022-03,1349,8,18,582638720
3,HCE,2022-04,1344,13,9,580540860
4,HCE,2022-05,1340,5,7,579585440
5,HCE,2022-06,1358,25,12,585809360
6,HCE,2022-07,1361,15,13,623287880
7,HCE,2022-08,1365,17,13,623745980
8,HCE,2022-09,1356,4,10,619505100
9,HCE,2022-10,1354,8,8,618599740


In [26]:
gdf.columns = ["약식명", "기준일자", "직원수", "신규취득", "자격상실", "당월고지금액"]

In [27]:
gdf["평균소득월액"] = gdf[["당월고지금액","직원수"]].astype('str').apply(get_평균소득월액, axis=1)

In [28]:
gdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   약식명     195 non-null    object
 1   기준일자    195 non-null    object
 2   직원수     195 non-null    int64 
 3   신규취득    195 non-null    int64 
 4   자격상실    195 non-null    int64 
 5   당월고지금액  195 non-null    int64 
 6   평균소득월액  195 non-null    int64 
dtypes: int64(5), object(2)
memory usage: 10.8+ KB


In [29]:
gdf.head()

,약식명,기준일자,직원수,신규취득,자격상실,당월고지금액,평균소득월액
0,HCE,2022-01,1331,19,15,557290480,4652228
1,HCE,2022-02,1357,41,16,566574100,4639106
2,HCE,2022-03,1349,8,18,582638720,4798935
3,HCE,2022-04,1344,13,9,580540860,4799444
4,HCE,2022-05,1340,5,7,579585440,4805849


In [30]:
요약통계 = gdf.groupby(['약식명'])[['직원수', '신규취득', '자격상실', '평균소득월액']].mean().round().reset_index()
요약통계

,약식명,직원수,신규취득,자격상실,평균소득월액
0,HCE,1337.0,16.0,19.0,4988296.0
1,HDI,2687.0,37.0,41.0,5168520.0
2,HDX,492.0,44.0,6.0,5115454.0
3,대동,1217.0,45.0,44.0,4405600.0
4,두산밥캣,139.0,4.0,3.0,5182349.0
5,두산산차,610.0,14.0,10.0,4741647.0
6,모비스,11109.0,125.0,68.0,5165160.0
7,모트롤,491.0,2.0,4.0,5164522.0
8,볼보코리아,1461.0,14.0,12.0,4997742.0
9,삼성전자,115622.0,1136.0,548.0,5282183.0


In [31]:
import pickle
with open ("pickle_df2.pickle", 'wb') as pickle_filename:
    pickle.dump(gdf, pickle_filename)